# Implementation of the "NULL labeling" defence mechanism

## 1. Training the classifier on clean images

## 2. Computing adversarial examples and `NULL` probabilities

## 3. Continue training on both clean and adversarial images

## 4. Benchmark on classical and adversarial examples